# Notebook 3: Preparación para BigQuery y Looker Studio

**Fundación Teletón - Análisis de Satisfacción de Benefactores**

---

## Objetivo

Este notebook prepara:
- Dataset optimizado para carga en BigQuery
- Esquema de tablas con tipos de datos correctos
- Campos calculados y métricas derivadas
- Documentación de dimensiones y métricas para Looker Studio
- Archivo CSV listo para importar a BigQuery

## Flujo de Trabajo

1. **Python (este notebook)** → Preparar y exportar datos
2. **BigQuery** → Cargar CSV y crear tablas
3. **Looker Studio** → Conectar y visualizar

---

## 1. Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Cargar Dataset Limpio

In [ ]:
# Cargar datos limpios del Notebook 1
df = pd.read_csv('../datos/teleton_limpio.csv')

print(f"📊 Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"\nColumnas: {list(df.columns)}")
print(f"\nTipos de datos:")
print(df.dtypes)

## 3. Optimizar Tipos de Datos para BigQuery

BigQuery requiere tipos de datos específicos. Optimizaremos el dataset para mejorar rendimiento y costos.

In [ ]:
# Crear copia del dataframe
df_bq = df.copy()

# Convertir fecha a formato ISO 8601 (YYYY-MM-DD)
if 'fecha' in df_bq.columns:
    df_bq['fecha'] = pd.to_datetime(df_bq['fecha'], errors='coerce')
    df_bq['fecha'] = df_bq['fecha'].dt.strftime('%Y-%m-%d')
    print("✅ Fecha convertida a formato ISO 8601")

# Asegurar que variables categóricas sean STRING
vars_categoricas = ['giro', 'puesto', 'estado', 'nps_categoria', 'segmento_antiguedad']
for col in vars_categoricas:
    if col in df_bq.columns:
        df_bq[col] = df_bq[col].astype(str)
        # Reemplazar 'nan' string por None (NULL en BigQuery)
        df_bq[col] = df_bq[col].replace('nan', None)

print("✅ Variables categóricas optimizadas como STRING")

# Convertir variables numéricas a FLOAT64
vars_numericas = ['anos_benefactor', 'satisfaccion_general', 'nps', 'calidad_percibida', 
                  'transparencia', 'indice_calidad_servicio']
for col in vars_numericas:
    if col in df_bq.columns:
        df_bq[col] = pd.to_numeric(df_bq[col], errors='coerce')

print("✅ Variables numéricas convertidas a FLOAT64")

# Variables Likert (1-5) como INTEGER
vars_likert = [
    'apariencia_comportamiento', 'claridad_procedimientos', 'cumplimiento_horarios',
    'conocimiento_competencia', 'claridad_primer_contacto', 'rapidez_respuesta',
    'disposicion_ayuda', 'flexibilidad', 'actitud_comprensiva',
    'tiempo_dedicado', 'preocupacion_situacion', 'atencion_personalizada',
    'comunicacion_continua'
]

for col in vars_likert:
    if col in df_bq.columns:
        df_bq[col] = pd.to_numeric(df_bq[col], errors='coerce')
        df_bq[col] = df_bq[col].round().astype('Int64')  # Int64 permite NaN

print("✅ Variables Likert convertidas a INTEGER")

print(f"\n📊 Dataset optimizado para BigQuery: {df_bq.shape}")

## 4. Crear Tabla de Dimensiones

Separamos las dimensiones (atributos descriptivos) de las métricas (valores numéricos).

In [ ]:
# Crear ID único para cada registro
df_bq['benefactor_id'] = range(1, len(df_bq) + 1)

# Tabla de dimensiones (perfil del benefactor)
dimensiones = df_bq[[
    'benefactor_id',
    'giro',
    'puesto',
    'estado',
    'anos_benefactor',
    'segmento_antiguedad',
    'fecha'
]].copy()

print("✅ Tabla de dimensiones creada")
print(f"   Dimensiones: {dimensiones.shape[0]} registros × {dimensiones.shape[1]} columnas")
print(f"\n   Columnas: {list(dimensiones.columns)}")
dimensiones.head(3)

## 5. Crear Tabla de Hechos (Facts)

Tabla principal con todas las métricas y evaluaciones.

In [ ]:
# Columnas de la tabla de hechos
columnas_hechos = ['benefactor_id'] + vars_likert + vars_numericas + ['nps_categoria']

# Filtrar solo columnas que existen
columnas_hechos = [col for col in columnas_hechos if col in df_bq.columns]

hechos = df_bq[columnas_hechos].copy()

print("✅ Tabla de hechos creada")
print(f"   Hechos: {hechos.shape[0]} registros × {hechos.shape[1]} columnas")
print(f"\n   Columnas: {list(hechos.columns)}")
hechos.head(3)

## 6. Crear Tabla de Agregaciones Pre-calculadas

Para mejorar el rendimiento de Looker Studio, pre-calculamos agregaciones comunes.

In [ ]:
# Agregación por Giro
agg_giro = df_bq.groupby('giro').agg({
    'benefactor_id': 'count',
    'satisfaccion_general': 'mean',
    'nps': 'mean',
    'indice_calidad_servicio': 'mean',
    'transparencia': 'mean',
    'anos_benefactor': 'mean'
}).rename(columns={'benefactor_id': 'cantidad_benefactores'}).round(2).reset_index()

agg_giro['dimension'] = 'giro'
agg_giro = agg_giro.rename(columns={'giro': 'dimension_valor'})

print("✅ Agregación por Giro:")
print(agg_giro.head())

# Agregación por Estado
agg_estado = df_bq.groupby('estado').agg({
    'benefactor_id': 'count',
    'satisfaccion_general': 'mean',
    'nps': 'mean',
    'indice_calidad_servicio': 'mean',
    'transparencia': 'mean',
    'anos_benefactor': 'mean'
}).rename(columns={'benefactor_id': 'cantidad_benefactores'}).round(2).reset_index()

agg_estado['dimension'] = 'estado'
agg_estado = agg_estado.rename(columns={'estado': 'dimension_valor'})

print("\n✅ Agregación por Estado (Top 5):")
print(agg_estado.sort_values('cantidad_benefactores', ascending=False).head())

# Agregación por Segmento de Antigüedad
agg_antiguedad = df_bq.groupby('segmento_antiguedad').agg({
    'benefactor_id': 'count',
    'satisfaccion_general': 'mean',
    'nps': 'mean',
    'indice_calidad_servicio': 'mean',
    'transparencia': 'mean',
    'anos_benefactor': 'mean'
}).rename(columns={'benefactor_id': 'cantidad_benefactores'}).round(2).reset_index()

agg_antiguedad['dimension'] = 'segmento_antiguedad'
agg_antiguedad = agg_antiguedad.rename(columns={'segmento_antiguedad': 'dimension_valor'})

print("\n✅ Agregación por Antigüedad:")
print(agg_antiguedad)

# Combinar todas las agregaciones en una sola tabla
agregaciones_todas = pd.concat([agg_giro, agg_estado, agg_antiguedad], ignore_index=True)

print(f"\n✅ Tabla de agregaciones combinada: {agregaciones_todas.shape}")

## 7. Crear Tabla de KPIs Globales

In [ ]:
# Calcular KPIs globales
total_benefactores = len(df_bq)
promotores = (df_bq['nps'] >= 9).sum()
pasivos = ((df_bq['nps'] >= 7) & (df_bq['nps'] <= 8)).sum()
detractores = (df_bq['nps'] <= 6).sum()
nps_score = ((promotores - detractores) / total_benefactores) * 100

kpis_globales = pd.DataFrame([
    {
        'kpi_nombre': 'Total Benefactores',
        'kpi_valor': total_benefactores,
        'kpi_unidad': 'cantidad',
        'kpi_categoria': 'general'
    },
    {
        'kpi_nombre': 'Satisfacción General',
        'kpi_valor': round(df_bq['satisfaccion_general'].mean(), 2),
        'kpi_unidad': '/10',
        'kpi_categoria': 'satisfaccion'
    },
    {
        'kpi_nombre': 'Net Promoter Score',
        'kpi_valor': round(nps_score, 1),
        'kpi_unidad': '%',
        'kpi_categoria': 'lealtad'
    },
    {
        'kpi_nombre': 'Promotores',
        'kpi_valor': promotores,
        'kpi_unidad': 'cantidad',
        'kpi_categoria': 'lealtad'
    },
    {
        'kpi_nombre': 'Pasivos',
        'kpi_valor': pasivos,
        'kpi_unidad': 'cantidad',
        'kpi_categoria': 'lealtad'
    },
    {
        'kpi_nombre': 'Detractores',
        'kpi_valor': detractores,
        'kpi_unidad': 'cantidad',
        'kpi_categoria': 'lealtad'
    },
    {
        'kpi_nombre': 'Índice Calidad Servicio',
        'kpi_valor': round(df_bq['indice_calidad_servicio'].mean(), 2),
        'kpi_unidad': '/5',
        'kpi_categoria': 'calidad'
    },
    {
        'kpi_nombre': 'Transparencia',
        'kpi_valor': round(df_bq['transparencia'].mean(), 2),
        'kpi_unidad': '/10',
        'kpi_categoria': 'satisfaccion'
    },
    {
        'kpi_nombre': 'Calidad Percibida',
        'kpi_valor': round(df_bq['calidad_percibida'].mean(), 2),
        'kpi_unidad': '/10',
        'kpi_categoria': 'satisfaccion'
    },
    {
        'kpi_nombre': 'Antigüedad Promedio',
        'kpi_valor': round(df_bq['anos_benefactor'].mean(), 1),
        'kpi_unidad': 'años',
        'kpi_categoria': 'general'
    }
])

print("✅ Tabla de KPIs globales creada:")
print(kpis_globales.to_string(index=False))

## 8. Generar Esquema BigQuery

Crear archivo JSON con el esquema de cada tabla para facilitar la carga a BigQuery.

In [ ]:
def generar_esquema_bigquery(df, nombre_tabla):
    """
    Genera esquema BigQuery en formato JSON a partir de un DataFrame
    """
    type_mapping = {
        'int64': 'INTEGER',
        'Int64': 'INTEGER',
        'float64': 'FLOAT',
        'object': 'STRING',
        'string': 'STRING',
        'datetime64[ns]': 'DATE',
        'bool': 'BOOLEAN'
    }
    
    schema = []
    for col in df.columns:
        dtype = str(df[col].dtype)
        bq_type = type_mapping.get(dtype, 'STRING')
        
        # Manejar fechas
        if col == 'fecha':
            bq_type = 'DATE'
        
        schema.append({
            'name': col,
            'type': bq_type,
            'mode': 'NULLABLE'
        })
    
    return schema


# Generar esquemas para cada tabla
esquema_dimensiones = generar_esquema_bigquery(dimensiones, 'dimensiones')
esquema_hechos = generar_esquema_bigquery(hechos, 'hechos')
esquema_agregaciones = generar_esquema_bigquery(agregaciones_todas, 'agregaciones')
esquema_kpis = generar_esquema_bigquery(kpis_globales, 'kpis_globales')

esquemas = {
    'dimensiones': esquema_dimensiones,
    'hechos': esquema_hechos,
    'agregaciones': esquema_agregaciones,
    'kpis_globales': esquema_kpis
}

print("✅ Esquemas BigQuery generados")
print(f"\n📋 Ejemplo - Esquema tabla 'dimensiones':")
for campo in esquema_dimensiones[:5]:
    print(f"   {campo['name']:30s} → {campo['type']:10s} ({campo['mode']})")

## 9. Exportar Datos para BigQuery

In [ ]:
import os

# Crear carpeta para exports BigQuery
os.makedirs('../looker/bigquery_data', exist_ok=True)

# Exportar cada tabla como CSV
dimensiones.to_csv('../looker/bigquery_data/dimensiones.csv', index=False, encoding='utf-8-sig')
hechos.to_csv('../looker/bigquery_data/hechos.csv', index=False, encoding='utf-8-sig')
agregaciones_todas.to_csv('../looker/bigquery_data/agregaciones.csv', index=False, encoding='utf-8-sig')
kpis_globales.to_csv('../looker/bigquery_data/kpis_globales.csv', index=False, encoding='utf-8-sig')

# Exportar dataset completo también (para análisis ad-hoc)
df_bq.to_csv('../looker/bigquery_data/teleton_completo.csv', index=False, encoding='utf-8-sig')

print("✅ Archivos CSV exportados a looker/bigquery_data/:")
print("   📄 dimensiones.csv")
print("   📄 hechos.csv")
print("   📄 agregaciones.csv")
print("   📄 kpis_globales.csv")
print("   📄 teleton_completo.csv")

# Exportar esquemas como JSON
with open('../looker/bigquery_schemas.json', 'w', encoding='utf-8') as f:
    json.dump(esquemas, f, indent=2, ensure_ascii=False)

print("\n✅ Esquemas exportados a looker/bigquery_schemas.json")

## 10. Crear Diccionario de Datos para Looker Studio

In [ ]:
diccionario_datos = {
    "proyecto": "Dashboard Fundación Teletón - Satisfacción de Benefactores",
    "fecha_creacion": datetime.now().strftime('%Y-%m-%d'),
    "tablas": {
        "dimensiones": {
            "descripcion": "Perfil y características de los benefactores",
            "tipo": "Dimensiones",
            "campos": [
                {"nombre": "benefactor_id", "tipo": "INTEGER", "descripcion": "ID único del benefactor"},
                {"nombre": "giro", "tipo": "STRING", "descripcion": "Giro o sector empresarial"},
                {"nombre": "puesto", "tipo": "STRING", "descripcion": "Puesto del encuestado"},
                {"nombre": "estado", "tipo": "STRING", "descripcion": "Estado de la República"},
                {"nombre": "anos_benefactor", "tipo": "FLOAT", "descripcion": "Años como benefactor de Teletón"},
                {"nombre": "segmento_antiguedad", "tipo": "STRING", "descripcion": "Segmento según antigüedad (Nuevo/Establecido/Veterano/Histórico)"},
                {"nombre": "fecha", "tipo": "DATE", "descripcion": "Fecha de la encuesta"}
            ]
        },
        "hechos": {
            "descripcion": "Evaluaciones y métricas de satisfacción",
            "tipo": "Hechos (Facts)",
            "metricas_principales": [
                {"nombre": "satisfaccion_general", "tipo": "FLOAT", "escala": "1-10", "descripcion": "Satisfacción general con Teletón"},
                {"nombre": "nps", "tipo": "FLOAT", "escala": "0-10", "descripcion": "Net Promoter Score (probabilidad de recomendar)"},
                {"nombre": "calidad_percibida", "tipo": "FLOAT", "escala": "1-10", "descripcion": "Calidad percibida del servicio"},
                {"nombre": "transparencia", "tipo": "FLOAT", "escala": "1-10", "descripcion": "Transparencia en el uso de recursos"},
                {"nombre": "indice_calidad_servicio", "tipo": "FLOAT", "escala": "1-5", "descripcion": "Promedio de 13 dimensiones de calidad"}
            ],
            "dimensiones_calidad": [
                {"nombre": "apariencia_comportamiento", "escala": "1-5", "descripcion": "Apariencia y comportamiento del promotor"},
                {"nombre": "claridad_procedimientos", "escala": "1-5", "descripcion": "Claridad de procedimientos y documentación"},
                {"nombre": "cumplimiento_horarios", "escala": "1-5", "descripcion": "Cumplimiento de horarios comprometidos"},
                {"nombre": "conocimiento_competencia", "escala": "1-5", "descripcion": "Conocimiento y competencia técnica"},
                {"nombre": "claridad_primer_contacto", "escala": "1-5", "descripcion": "Claridad en el primer contacto"},
                {"nombre": "rapidez_respuesta", "escala": "1-5", "descripcion": "Rapidez de respuesta a solicitudes"},
                {"nombre": "disposicion_ayuda", "escala": "1-5", "descripcion": "Disposición para ayudar"},
                {"nombre": "flexibilidad", "escala": "1-5", "descripcion": "Flexibilidad ante cambios"},
                {"nombre": "actitud_comprensiva", "escala": "1-5", "descripcion": "Actitud comprensiva y empática"},
                {"nombre": "tiempo_dedicado", "escala": "1-5", "descripcion": "Tiempo dedicado a atender necesidades"},
                {"nombre": "preocupacion_situacion", "escala": "1-5", "descripcion": "Preocupación por la situación del benefactor"},
                {"nombre": "atencion_personalizada", "escala": "1-5", "descripcion": "Atención personalizada"},
                {"nombre": "comunicacion_continua", "escala": "1-5", "descripcion": "Comunicación continua y seguimiento"}
            ]
        },
        "agregaciones": {
            "descripcion": "Métricas pre-calculadas por dimensión",
            "tipo": "Agregaciones",
            "campos": [
                {"nombre": "dimension", "tipo": "STRING", "descripcion": "Tipo de dimensión (giro/estado/segmento_antiguedad)"},
                {"nombre": "dimension_valor", "tipo": "STRING", "descripcion": "Valor de la dimensión"},
                {"nombre": "cantidad_benefactores", "tipo": "INTEGER", "descripcion": "Cantidad de benefactores en este segmento"},
                {"nombre": "satisfaccion_general", "tipo": "FLOAT", "descripcion": "Promedio de satisfacción general"},
                {"nombre": "nps", "tipo": "FLOAT", "descripcion": "Promedio de NPS"},
                {"nombre": "indice_calidad_servicio", "tipo": "FLOAT", "descripcion": "Promedio de índice de calidad"},
                {"nombre": "transparencia", "tipo": "FLOAT", "descripcion": "Promedio de transparencia"},
                {"nombre": "anos_benefactor", "tipo": "FLOAT", "descripcion": "Promedio de años como benefactor"}
            ]
        },
        "kpis_globales": {
            "descripcion": "KPIs consolidados a nivel global",
            "tipo": "KPIs",
            "campos": [
                {"nombre": "kpi_nombre", "tipo": "STRING", "descripcion": "Nombre del KPI"},
                {"nombre": "kpi_valor", "tipo": "FLOAT", "descripcion": "Valor del KPI"},
                {"nombre": "kpi_unidad", "tipo": "STRING", "descripcion": "Unidad de medida"},
                {"nombre": "kpi_categoria", "tipo": "STRING", "descripcion": "Categoría del KPI (general/satisfaccion/lealtad/calidad)"}
            ]
        }
    },
    "metricas_calculadas_looker": [
        {
            "nombre": "NPS Score (%)",
            "formula": "((COUNT(CASE WHEN nps >= 9 THEN 1 END) - COUNT(CASE WHEN nps <= 6 THEN 1 END)) / COUNT(*)) * 100",
            "descripcion": "Net Promoter Score calculado dinámicamente"
        },
        {
            "nombre": "% Promotores",
            "formula": "(COUNT(CASE WHEN nps >= 9 THEN 1 END) / COUNT(*)) * 100",
            "descripcion": "Porcentaje de promotores (NPS 9-10)"
        },
        {
            "nombre": "% Detractores",
            "formula": "(COUNT(CASE WHEN nps <= 6 THEN 1 END) / COUNT(*)) * 100",
            "descripcion": "Porcentaje de detractores (NPS 0-6)"
        },
        {
            "nombre": "Satisfacción Normalizada (%)",
            "formula": "(AVG(satisfaccion_general) / 10) * 100",
            "descripcion": "Satisfacción general normalizada a escala 0-100%"
        }
    ],
    "paleta_colores_teleton": {
        "principal": {
            "amarillo": "#F7C600",
            "morado_profundo": "#4B1F76",
            "morado_medio": "#7E3AA7"
        },
        "acentos": {
            "magenta": "#D7268F",
            "azul": "#1A2A6C",
            "naranja": "#F39C12",
            "verde": "#2ECC71",
            "rojo": "#E74C3C"
        },
        "neutros": {
            "blanco": "#FFFFFF",
            "gris_claro": "#F5F5F5",
            "gris_medio": "#7F8C8D",
            "gris_oscuro": "#2D3436"
        }
    }
}

# Exportar diccionario
with open('../looker/diccionario_datos.json', 'w', encoding='utf-8') as f:
    json.dump(diccionario_datos, f, indent=2, ensure_ascii=False)

print("✅ Diccionario de datos exportado a looker/diccionario_datos.json")
print("\n📚 Contiene:")
print("   - Descripción de 4 tablas")
print("   - Definición de campos y tipos de datos")
print("   - 4 métricas calculadas sugeridas para Looker")
print("   - Paleta de colores Teletón")

## 11. Validación de Datos Exportados

In [ ]:
# Validar que los archivos se crearon correctamente
import os

archivos_esperados = [
    '../looker/bigquery_data/dimensiones.csv',
    '../looker/bigquery_data/hechos.csv',
    '../looker/bigquery_data/agregaciones.csv',
    '../looker/bigquery_data/kpis_globales.csv',
    '../looker/bigquery_data/teleton_completo.csv',
    '../looker/bigquery_schemas.json',
    '../looker/diccionario_datos.json'
]

print("🔍 Validando archivos exportados...\n")

for archivo in archivos_esperados:
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo)
        tamaño_kb = tamaño / 1024
        print(f"   ✅ {archivo.split('/')[-1]:30s} → {tamaño_kb:>8.1f} KB")
    else:
        print(f"   ❌ {archivo.split('/')[-1]:30s} → NO ENCONTRADO")

print("\n✅ Validación completada")

## 12. Estadísticas Finales

In [ ]:
print("="*70)
print("RESUMEN: Preparación BigQuery y Looker Studio")
print("="*70)

print("\n📊 TABLAS CREADAS:")
print(f"   1. dimensiones         → {dimensiones.shape[0]:>4} registros × {dimensiones.shape[1]:>2} columnas")
print(f"   2. hechos              → {hechos.shape[0]:>4} registros × {hechos.shape[1]:>2} columnas")
print(f"   3. agregaciones        → {agregaciones_todas.shape[0]:>4} registros × {agregaciones_todas.shape[1]:>2} columnas")
print(f"   4. kpis_globales       → {kpis_globales.shape[0]:>4} registros × {kpis_globales.shape[1]:>2} columnas")

print("\n📈 KPIS PRINCIPALES:")
for idx, row in kpis_globales.iterrows():
    print(f"   • {row['kpi_nombre']:30s} → {row['kpi_valor']:>8.2f} {row['kpi_unidad']}")

print("\n📦 ARCHIVOS EXPORTADOS:")
print("   ✅ 5 archivos CSV (tablas para BigQuery)")
print("   ✅ bigquery_schemas.json (esquemas de tablas)")
print("   ✅ diccionario_datos.json (documentación completa)")

print("\n🎯 PRÓXIMOS PASOS:")
print("   1. Subir CSVs a BigQuery")
print("   2. Configurar permisos IAM para @tec.mx")
print("   3. Conectar Looker Studio a BigQuery")
print("   4. Crear dashboard siguiendo la paleta Teletón")

print("="*70)